In [1]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("trongnguyen04/nckh-2425-crops")
print("Path to dataset files:", path)

SRC=path
DEST="/content/dataset_original/crops"
!mkdir -p $(dirname $DEST)
!cp -r "$SRC" "$DEST"

100%|██████████| 46.8M/46.8M [00:03<00:00, 14.9MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/trongnguyen04/nckh-2425-crops/versions/3


In [2]:
!git clone https://github.com/htrnguyen/compare_ocr_benchmark.git

Cloning into 'compare_ocr_benchmark'...
remote: Enumerating objects: 91, done.
remote: Counting objects: 100% (91/91), done.
remote: Compressing objects: 100% (60/60), done.
remote: Total 91 (delta 41), reused 75 (delta 25), pack-reused 0 (from 0)
Receiving objects: 100% (91/91), 4.54 MiB | 18.76 MiB/s, done.
Resolving deltas: 100% (41/41), done.


In [3]:
!pip install python-Levenshtein jiwer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.7/161.7 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 65.4 MB/s eta 0:00:00


In [4]:
import sys
sys.path.append('/content/compare_ocr_benchmark/common')

from image_preprocessing import preprocess_image
from ocr_base import OCRModelBase
from metrics import compute_metrics
from utils import read_annotations, save_results

import os
import pandas as pd
import time

In [5]:
!pip install easyocr

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 80.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 68.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 36.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 51.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 108.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 422.8/422.8 kB 39.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [6]:
import easyocr
import numpy as np

class EssayOCRModel(OCRModelBase):
    def __init__(self, config=None):
        super().__init__(config)
        self.reader = easyocr.Reader(['vi', 'en'], gpu=True)

    def predict(self, img_pil, img_np=None):
        t1 = time.perf_counter()
        if img_np is None:
            img_np = np.array(img_pil)
        result = self.reader.readtext(img_np, detail=0, paragraph=True)
        if isinstance(result, list):
            result_str = " ".join(result).strip()
        else:
            result_str = str(result).strip()
        t2 = time.perf_counter()
        return result_str, t2 - t1

In [7]:
CSV_ANN = '/content/dataset_original/crops/crops_gt.csv'
IMG_DIR = '/content/dataset_original/crops'

df = read_annotations(CSV_ANN)

In [8]:
model = EssayOCRModel()

results = []
for idx, row in df.iterrows():
    fname = row['filename']
    desc_gt = row['description_gt']
    label = row.get('label', '')
    img_path = os.path.join(IMG_DIR, fname)

    img_pil, img_np = preprocess_image(img_path, target_size=448, do_auto=True)
    pred, infer_time = model.predict(img_pil, img_np)
    metrics = compute_metrics(desc_gt, pred)

    results.append({
        "filename": fname,
        "label": label,
        "ground_truth": desc_gt,
        "predicted_text": pred,
        "cer": metrics["cer"],
        "wer": metrics["wer"],
        "lev": metrics["lev"],
        "acc": metrics["acc"],
        "time": round(infer_time, 3)
    })

Progress: |██████████████████████████████████████████████████| 100.0% Complete

Progress: |██████████████████████████████████████████████████| 100.0% Complete

In [9]:
import os
OUT_CSV = '/content/compare_ocr_benchmark/results/essayocr_results.csv'
os.makedirs(os.path.dirname(OUT_CSV), exist_ok=True)
save_results(results, OUT_CSV)

Lưu thành công: /content/compare_ocr_benchmark/results/essayocr_results.csv
